In [ ]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
def getBusStop(df, spectag):
    tagdf = df.find_all(spectag)
    returndf = []
    for tag in tagdf:
        temp = tag.text.strip() # takes text in between each tag
        returndf.append(temp) # append it to "date" list
    returndf = pd.Series(returndf)
    return returndf

routesurl = "http://www.ctabustracker.com/bustime/api/v3/getroutes?key=kUXKRKMhBsjB3a32q5UcLZ4YX"
routes = requests.get(url=routesurl)
routesdata = BeautifulSoup(routes.text,"xml")
routetag = routesdata.find_all("rt")
routenumbers = []
for tag in routetag: # checks every tag in "date_tag"
    temp = tag.text.strip() # takes text in between each tag
    routenumbers.append(temp) # append it to "date" list
routenumbers = pd.Series(routenumbers) # converts list to a series
templateurl = "https://www.ctabustracker.com/bustime/api/v3/getdirections?key=kUXKRKMhBsjB3a32q5UcLZ4YX&rt="
templateurl2 = "https://www.ctabustracker.com/bustime/api/v3/getstops?key=kUXKRKMhBsjB3a32q5UcLZ4YX&rt="
templateurl3 = "&dir="
stopsloc = pd.DataFrame(columns=["StopId","Lat","Lon"])
for i in routenumbers:
    directionurl = "".join([templateurl, i])
    direction = requests.get(url=directionurl)
    directiondata = BeautifulSoup(direction.text,"xml")
    directiontag = directiondata.find_all("id")
    dir = []
    for tag in directiontag: # checks every tag in "date_tag"
        temp = tag.text.strip() # takes text in between each tag
        dir.append(temp) # append it to "date" list
    dir = pd.Series(dir) # converts list to a series
    stopurl = "".join([templateurl2, i])
    stopurl = "".join([stopurl, templateurl3])
    for w in dir:
        stopurl = "".join([stopurl, w])
        stops = requests.get(url=stopurl)
        stopsdata = BeautifulSoup(stops.text, "xml")
        StopId = getBusStop(stopsdata, "stpid")
        Lat = getBusStop(stopsdata, "lat")
        Lon = getBusStop(stopsdata, "lon")
        stops_df = pd.DataFrame({'StopId': StopId, "Lat": Lat, "Lon": Lon})
        stopsloc = pd.concat([stopsloc, stops_df], axis =0)




In [68]:
import os
os.getcwd()
ChicagoStops = stopsloc
DCStops = pd.read_csv("Metro_Bus_Stops.csv")
DCStops.columns=["StopId","Lon","Lat"]
LAStops = pd.read_csv("stops.txt")
LAStops = LAStops[["stop_id","stop_lat","stop_lon"]]
LAStops.columns={"StopId","Lon","Lat"}
ChicagoStops = ChicagoStops[["StopId","Lon","Lat"]]
ChicagoStops["City"] = "Chicago"
DCStops["City"] = "DC"
LAStops["City"] = "Los Angeles"
BusStops = pd.concat([ChicagoStops,LAStops,DCStops],axis=0)


In [70]:
BusStops

,StopId,Lon,Lat,City
0,8590,-87.846407,41.974865,Chicago
1,14143,-87.839028,41.983680,Chicago
2,9751,-87.836433,41.966700,Chicago
3,8566,-87.836837,41.973820,Chicago
4,14564,-87.688712,41.917725,Chicago
...,...,...,...,...
10358,NaN,-77.217035,38.825841,DC
10359,NaN,-76.889212,38.887514,DC
10360,NaN,-77.204532,38.831705,DC
10361,NaN,-77.193622,38.830114,DC
